In [1]:
from data_science_toolkit.scraping.scraping import *
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
# search each link 
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import math
import re
import time
# lets open booking:

dfolder='./downloads'
geko_path='C:/Users/jaoss/Downloads/geckodriver/geckodriver.exe'
link='https://www.booking.com/index.en.html'


# browser=start_up(dfolder=dfolder,link=link,geko_path=geko_path)



In [28]:
def ffx_preferences(dfolder, download=False):
    '''
    Sets the preferences of the firefox browser: download path.
    '''
    profile = webdriver.FirefoxProfile()
    # set download folder:
    profile.set_preference("browser.download.dir", dfolder)
    profile.set_preference("browser.download.folderList", 2)
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    profile.set_preference("browser.helperApps.neverAsk.saveToDisk",
                           "application/msword,application/rtf, application/csv,text/csv,image/png ,image/jpeg, application/pdf, text/html,text/plain,application/octet-stream")
    
    #profile.add_extension('/Users/luisignaciomenendezgarcia/Dropbox/CLASSES/class_bse_text_mining/class_scraping_bse/booking/booking/ublock_origin-1.55.0.xpi')


    # this allows to download pdfs automatically
    if download:
        profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf,application/x-pdf")
        profile.set_preference("pdfjs.disabled", True)

    options = Options()
    options.profile = profile
    return options


def start_up(link, dfolder, geko_path, firefox_binary_path, download=True):
    os.makedirs(dfolder, exist_ok=True)

    options = ffx_preferences(dfolder, download)
    options.binary_location = firefox_binary_path  # Set Firefox binary location
    service = Service(geko_path)
    browser = webdriver.Firefox(service=service, options=options)
   
    # Enter the website address here
    browser.get(link)
   
    time.sleep(5)  # Adjust sleep time as needed
    return browser


def check_and_click(browser, xpath, type):
    '''
    Function that checks whether the object is clickable and, if so, clicks on
    it. If not, waits one second and tries again.
    '''
    ck = False
    ss = 0
    while ck == False:
        ck = check_obscures(browser, xpath, type)
        time.sleep(1)
        ss += 1
        if ss == 15:
            # warn_sound()
            # return NoSuchElementException
            ck = True
            # browser.quit()

def check_obscures(browser, xpath, type):
    '''
    Function that checks whether the object is being "obscured" by any element so
    that it is not clickable. Important: if True, the object is going to be clicked!
    '''
    try:
        if type == "xpath":
            browser.find_element('xpath',xpath).click()
        elif type == "id":
            browser.find_element('id',xpath).click()
        elif type == "css":
            browser.find_element('css selector',xpath).click()
        elif type == "class":
            browser.find_element('class name',xpath).click()
        elif type == "link":
            browser.find_element('link text',xpath).click()
    except (ElementClickInterceptedException, NoSuchElementException, StaleElementReferenceException) as e:
        print(e)
        return False
    return True

dfolder='C:/Users/jaoss/Desktop/Data_Science/2nd_Term/Text_Mining/Assignment_1/booking_scraping/Downloads'
geko_path='C:/Users/jaoss/Downloads/geckodriver/geckodriver.exe'
link='https://www.booking.com'
firefox_binary_path = r'C:/Program Files/Mozilla Firefox/firefox.exe'  # Example path, adjust as needed


browser=start_up(dfolder=dfolder,link=link,geko_path=geko_path, firefox_binary_path=firefox_binary_path)

In [3]:
def get_number_pages(browser):
    '''
    Get the number of pages. 
    '''
    a = browser.find_elements('xpath','//div[@class="d8f77e681c"]')
    #number of total hotel listings divided of listings p. page
    total_listings = int(re.findall(r'\d+', a[-1].text)[0])
    page_num = math.floor(total_listings / 25)
    print(f'Total hotels: {total_listings} \nNumber of pages: {page_num}')
    return(page_num) 

In [27]:
def search_hotels(start_date, end_date, place, browser):
    browser.get('https://www.booking.com')
    # Clear browsing history and cookies
    browser.delete_all_cookies()
    xpath = '//*[@id="onetrust-accept-btn-handler"]'

    time.sleep(10)
    try:
        check_and_click(browser, xpath, type='xpath')
    except:
        pass
    time.sleep(5)

    #accept cookies, refresh page then exit pop up (if it appears)
    browser.refresh()
    browser.refresh()
    browser.refresh()
    time.sleep(10)
    try:
        check_and_click(browser, '//button[@aria-label="Dismiss sign-in info."]', type='xpath')
    except:
    # Handle the exception without displaying the error message
        pass
    
    try:
        # Switch to the iframe and exit the google long in pop up
        iframe_path  = (By.XPATH, '//iframe[@title="Cuadro de diálogo Iniciar sesión con Google"]')
        iframe = WebDriverWait(browser, 10).until(EC.frame_to_be_available_and_switch_to_it(iframe_path))

        element_locator = (By.XPATH, '//div[@aria-label="Cerrar"]')
        element = WebDriverWait(browser, 10).until(EC.element_to_be_clickable(element_locator))
        element.click()

        browser.switch_to.default_content()
    except Exception as e:
        pass

        
    search1 = browser.find_element(by='xpath',value='//*[@id=":re:"]')
    search1.send_keys(place)

    #click date selector

    css='button.ebbedaf8ac:nth-child(2) > span:nth-child(1)'

    browser.find_element('css selector',css).click()


    #click start and end date

    start_date_found = False
    end_date_found = False
    while start_date_found == False or end_date_found == False:
        path='//span[@class="cf06f772fa"]'
        dates = browser.find_elements('xpath',path)
        for date in dates:
            if date.get_attribute("data-date") == start_date:
                date.click()
                start_date_found = True
            if date.get_attribute("data-date") == end_date:
                date.click()
                end_date_found = True
                break
        next_page_button = '//button[@class="a83ed08757 c21c56c305 f38b6daa18 d691166b09 f671049264 deab83296e f4552b6561 dc72a8413c f073249358"]'
        check_and_click(browser, next_page_button, type='xpath')
        time.sleep(3)

    time.sleep(3)

    #click search button
    my_xpath='/html/body/div[3]/div[2]/div/form/div[1]/div[4]/button'

    check_obscures(browser,my_xpath , type='xpath')
    check_and_click(browser,my_xpath , type='xpath')

    hotel_box_path = '//div[@data-testid="filters-group-label-content" and contains(text(), "Hotels")]'
    # hotel_box_path = '/html/body/div[4]/div/div[2]/div/div[2]/div[3]/div[1]/div[3]/div[4]/div[8]/label/span[3]/div/div'
    hotel_boxes = browser.find_elements('xpath', hotel_box_path)

    if hotel_boxes:
        hotel_boxes[0].click()
    else:
        print("Element not found")
    time.sleep(5)

    # # Change language to english
    # lang_button = 'div.aca0ade214:nth-child(2) > span:nth-child(2) > button:nth-child(1)'
    # eng_button = 'div.ebac6e22e9 > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > ul:nth-child(1) > li:nth-child(3) > button:nth-child(1)'
    # browser.find_element('css selector',lang_button).click()
    # time.sleep(2)
    # browser.find_element('css selector',eng_button).click()
    # time.sleep(5)  # wait for language change to take effect

    pages = get_number_pages(browser)
    print(pages)

    hotel_card_container = '//div[@data-testid="property-card"]'  # XPath expression to locate the hotel card container

    #check "Hotels" box

    

        


    data = []  # List to store the extracted data for each hotel


    for page in range(pages+1):
        hotels = browser.find_elements('xpath', hotel_card_container)  # Find all hotel elements on each iteration
        for hotel in hotels:  # Iterate over each hotel element
            data_dic = {}  # Dictionary to store the extracted data for the current hotel

            try:
                # Extract the photo URL of the hotel
                photo_path = './/img[@data-testid="image"]'  # XPath expression to locate the photo element
                photo_url = hotel.find_element('xpath', photo_path).get_attribute('src')  # Get the src attribute of the photo element
                data_dic['photo_url'] = photo_url  # Store the photo URL in the dictionary
            except NoSuchElementException:
                print('No photo URL')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the link of the hotel
                link_path = './/a[@class="a78ca197d0"]'  # XPath expression to locate the link element
                link = hotel.find_element('xpath', link_path).get_attribute('href')  # Get the href attribute of the link element
                data_dic['link'] = link  # Store the link in the dictionary
            except NoSuchElementException:
                print('No link')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the name of the hotel
                name_path = './/div[@data-testid="title"]'  # XPath expression to locate the name element
                name = hotel.find_element('xpath', name_path).text  # Get the text content of the name element
                data_dic['name'] = name  # Store the name in the dictionary
            except NoSuchElementException:
                print('No name')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the central distance of the hotel
                central_distance_path = './/span[@data-testid="distance"]'  # XPath expression to locate the central distance element
                central_distance = hotel.find_element('xpath', central_distance_path).text  # Get the text content of the central distance element
                data_dic['central_distance'] = central_distance  # Store the central distance in the dictionary
            except NoSuchElementException:
                print('No central distance')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the sustainability information of the hotel
                sustainability_path = './/span[@class="abf093bdfe d068504c75 f68ecd98ea"]'  # XPath expression to locate the sustainability element
                sustainability = hotel.find_element('xpath', sustainability_path).text  # Get the text content of the sustainability element
                data_dic['sustainability'] = sustainability  # Store the sustainability information in the dictionary
            except NoSuchElementException:
                data_dic['sustainability'] = None
                print('No sustainability information')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the rating of the hotel
                rating_path = './/div[@class="a3b8729ab1 d86cee9b25"]'  # XPath expression to locate the rating element
                rating = hotel.find_element('xpath', rating_path).text  # Get the text content of the rating element
                data_dic['rating'] = rating  # Store the rating in the dictionary
            except NoSuchElementException:
                print('No rating')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the comfort rating of the hotel
                comfort_rating_path = './/span[@class="a3332d346a"]'  # XPath expression to locate the comfort rating element
                comfort_rating = hotel.find_element('xpath', comfort_rating_path).text  # Get the text content of the comfort rating element
                data_dic['comfort_rating'] = comfort_rating  # Store the comfort rating in the dictionary
            except NoSuchElementException:
                print('No comfort rating')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the review count of the hotel
                review_count_path = './/div[@class="abf093bdfe f45d8e4c32 d935416c47"]'  # XPath expression to locate the review count element
                review_count = hotel.find_element('xpath', review_count_path).text  # Get the text content of the review count element
                data_dic['review_count'] = review_count  # Store the review count in the dictionary
            except NoSuchElementException:
                print('No review count')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the room description of the hotel
                room_description_path = './/h4[@class="abf093bdfe e8f7c070a7"]'  # XPath expression to locate the room description element
                room_description = hotel.find_element('xpath', room_description_path).text  # Get the text content of the room description element
                data_dic['room_description'] = room_description  # Store the room description in the dictionary
            except NoSuchElementException:
                print('No room description')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the room specifics of the hotel
                room_specifics_path = './/div[@class="abf093bdfe"]'  # XPath expression to locate the room specifics elements
                room_specifics = hotel.find_elements('xpath', room_specifics_path)  # Find all room specifics elements
                room_specifics_text = []
                for room_specific in room_specifics:
                    room_specifics_text.append(room_specific.text)  # Get the text content of each room specifics element
                data_dic['room_specifics'] = ' '.join(room_specifics_text)  # Store the room specifics as a single string in the dictionary
            except NoSuchElementException:
                print('No room specifics')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the price of the hotel
                price_path = './/span[@data-testid="price-and-discounted-price"]'  # XPath expression to locate the price element
                price = hotel.find_element('xpath', price_path).text  # Get the text content of the price element
                data_dic['price'] = price  # Store the price in the dictionary
            except NoSuchElementException:
                print('No price')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the original price of the hotel (if available)
                original_price_path = './/span[@class="c73ff05531 e84eb96b1f"]'  # XPath expression to locate the original price element
                original_price = hotel.find_element('xpath', original_price_path).text  # Get the text content of the original price element
                data_dic['original_price'] = original_price  # Store the original price in the dictionary
            except NoSuchElementException:
                data_dic['original_price'] = None  # If the original price element is not found, store None in the dictionary
                print('No original price')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the neighborhood and city of the hotel
                neighborhood_city_path = './/span[@data-testid="address"]'  # XPath expression to locate the neighborhood and city element
                neighborhood_city = hotel.find_element('xpath', neighborhood_city_path).text  # Get the text content of the neighborhood and city element
                data_dic['neighborhood_city'] = neighborhood_city  # Store the neighborhood and city in the dictionary
            except NoSuchElementException:
                print('No neighborhood and city')
            except Exception as e:
                print(e)
                print('Something went wrong')

            data.append(data_dic)  # Add the dictionary to the data list
            data_dic = {}
        # The data list now contains the extracted data for all hotels
        next_page_path = '//button[@aria-label="Next page"]'
        next_page = browser.find_elements('xpath', next_page_path)
        print(next_page)
        #element = WebDriverWait(browser, 30).until(EC.element_to_be_clickable(next_page[0]))
        next_page[0].click()
        time.sleep(3)
    data = pd.DataFrame(data)
    data['start_date'] = start_date
    data['end_date'] = end_date
    return data
        

In [5]:
def get_hotel_details(data):
    hotels_data = []
    x = 0 #just to tell how far the process is
    for link in data.link:
        hotel_data = {}
        print(link)
        x += 1
        print(f'{x} out of {len(data)}')
        r = requests.get(link)
        soup = BeautifulSoup(r.content, 'html.parser')
        id="hotel_sidebar_static_map"

        map_element = soup.find('a', id='hotel_sidebar_static_map').get('data-atlas-latlng')
        hotel_data['map_lat'] = map_element.split(',')[0]
        hotel_data['map_lng'] = map_element.split(',')[1]

        hotel_data['name'] = soup.find('h2', class_='d2fee87262 pp-header__title').get_text()
        #hotel_data['address'] = soup.find('span', class_='hp_address_subtitle').get_text()
        hotel_data['description'] = soup.find('div', class_='hp-description').get_text()

        # room_services = soup.find_all('span', class_='a5a5a75131')
        # hotel_data['room_services'] = ', '.join([room_service.get_text() for room_service in room_services])
        
        # room_ratings = soup.find_all('div', class_='ccb65902b2')
        # room_ratings_desc = [element.get_text() for element in room_ratings if ',' not in element.get_text()]
        # room_ratings = [element.get_text() for element in room_ratings if ',' in element.get_text()]


        # for index in range(0,len(room_ratings)): 
        #     hotel_data[f'room_rating_{room_ratings_desc[index]}'] = room_ratings[index]

    
        #hotel_data['room_options_count'] = len(soup.find_all('tr', class_='js-rt-block-row'))

        hotels_data.append(hotel_data)
    hotels_data = pd.DataFrame(hotels_data)
    return pd.concat([data, hotels_data], axis=1)


In [ ]:
start_date = '2024-03-03'
end_date = '2024-03-10'
place = "Los Ángeles, California, Estados Unidos"
file_path = start_date +'_'+end_date+'_'+place+'.csv'
data_c1_w1 = search_hotels(browser=browser, start_date=start_date,end_date=end_date,place=place)
data_c1_w1 = get_hotel_details(data_c1_w1)
data_c1_w1.to_csv(file_path)

In [ ]:
start_date = '2024-04-14'
end_date = '2024-04-21'
place = "Los Ángeles, California, Estados Unidos"
file_path = start_date +'_'+end_date+'_'+place+'.csv'
data_c1_w2 = search_hotels(browser=browser, start_date=start_date,end_date=end_date,place=place)
data_c1_w2 = get_hotel_details(data_c1_w2)

In [ ]:
start_date = '2024-03-03'
end_date = '2024-03-10'
place = "San Francisco, California, United States"
file_path = start_date +'_'+end_date+'_'+place+'.csv'
data_c2_w1 = search_hotels(browser=browser, start_date=start_date,end_date=end_date,place=place)
data_c2_w1 = get_hotel_details(data_c2_w1)

In [ ]:
start_date = '2024-04-14'
end_date = '2024-04-21'
place = "San Francisco, California, United States"
file_path = start_date +'_'+end_date+'_'+place+'.csv'
data_c2_w2 = search_hotels(browser=browser, start_date=start_date,end_date=end_date,place=place)
data_c2_w2 = get_hotel_details(data_c2_w2)

In [10]:
def clean_df(df):
  # Remove strings and symbols from the data
  df['central_distance'] = pd.to_numeric(df['central_distance'].str.replace(r'[a-zA-Z]','', regex=True))
  df['comfort_rating'] = pd.to_numeric(df['comfort_rating'].str.replace(r'[a-zA-Z]','', regex=True))
  df['sustainability'] = pd.to_numeric(df['sustainability'].str.replace(r'[a-zA-Z+]','', regex=True))
  df['review_count'] = pd.to_numeric(df['review_count'].str.replace(r'[a-zA-Z+,]','', regex=True))
  df['price'] = pd.to_numeric(df['price'].str.replace(r'[a-zA-Z€,]','', regex=True))
  df['original_price'] = pd.to_numeric(df['original_price'].str.replace(r'[a-zA-Z€,]','', regex=True))

  # Change date type data
  df['start_date'] = pd.to_datetime(df['start_date'])
  df['start_date'] = df['start_date'].dt.strftime('%Y/%m/%d')
  df['end_date'] = pd.to_datetime(df['end_date'])
  df['end_date'] = df['end_date'].dt.strftime('%Y/%m/%d')
    
  return df

# Function to remove the discount info
def remove_info(text):
    # Define the regex pattern to match the text to be removed
    pattern1 = re.compile(r"You're eligible for a Genius discount.*?all you have to do is sign in\.")
    pattern2 = re.compile(r'Couples in particular like the location –.*?Show me more\n\n\n\n', re.DOTALL)

    # Replace the matched portion with an empty string
    cleaned_text = re.sub(pattern1, '', text)
    cleaned_text = re.sub(pattern2, '', cleaned_text)
    cleaned_text = re.sub(r'\n+', '\n', cleaned_text)
    cleaned_text = cleaned_text.strip('\n')
    
    return cleaned_text

In [3]:
data_c1_w1 = clean_df(data_c1_w1)
data_c1_w1['description'] = data_c1_w1['description'].apply(remove_info)
data_c1_w2 = clean_df(data_c1_w2)
data_c1_w2['description'] = data_c1_w2['description'].apply(remove_info)
data_c2_w1 = clean_df(data_c2_w1)
data_c2_w1['description'] = data_c2_w1['description'].apply(remove_info)
data_c2_w2 = clean_df(data_c2_w2)
data_c2_w2['description'] = data_c2_w2['description'].apply(remove_info)

data_c1_w1.to_csv('2024-03-03_2024-03-10_LA_cleaned.csv')
data_c1_w2.to_csv('2024-03-03_2024-03-10_SF_cleaned.csv')
data_c2_w1.to_csv('2024-04-14_2024-04-21_LA_cleaned.csv')
data_c2_w2.to_csv('2024-04-14_2024-04-21_SF_cleaned.csv')



In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')


In [91]:
data_copy = pd.read_csv('2024-03-03_2024-03-10_Los Ángeles, California, Estados Unidos.csv')
data_copy = clean_df(data_copy)
data_copy['description'] = data_copy['description'].apply(remove_info)

sentences = data_copy['description'].tolist()

sentences

['Hollywood Homes minutes to everything SPACIOUS AND FREE PARKING is a sustainable condo hotel in Los Angeles, 2 miles from Capitol Records Building. This property encourages guests to embrace nature with its eco-friendly arrangements. This condo hotel offers free private parking, a 24-hour front desk, and free Wifi. The condo hotel has private entrance.\nAt the condo hotel, units are equipped with air conditioning, a seating area, a flat-screen TV with streaming services, a kitchen, a dining area, and a private bathroom with a hair dryer, a bath, and free toiletries. A dishwasher, an oven, and microwave are also offered, as well as a coffee machine and a kettle. At the condo hotel, the units come with bed linen and towels.\nThe condo hotel has a picnic area where you can spend the day outdoors.\nDolby Theater is 2.9 miles from the condo hotel, while Hollywood Bowl is 3.1 miles from the property. The nearest airport is Hollywood Burbank Airport, 8.7 miles from Hollywood Homes minutes t

In [ ]:
from nltk.corpus import stopwords

def strip(word):
    mod_string = re.sub(r'\W+', '', word)
    return mod_string

def abbr_or_lower(word):
    if re.match('([A-Z]+[a-z]*){2,}', word):
        return word
    else:
        return word.lower()
    
corpus_stop=[]
for index, row in enumerate(sentences):
    words = row.split()
    lowered = [abbr_or_lower(strip(w)) for w in words if abbr_or_lower(strip(w)) not in set(stopwords.words('english'))]
    lowered_sentence = ' '.join(lowered)
    print(lowered_sentence)
    corpus_stop.append(lowered)
    
    # Replace the original sentence with the lowercased version in the 'sentences' list
    sentences[index] = lowered_sentence

sentences

In [94]:
from nltk.stem.snowball import SnowballStemmer
englishStemmer=SnowballStemmer("english")

corpus_stem=[]
for sentence in sentences:
    words = sentence.split()
    lowered = [englishStemmer.stem(strip(w)) for w in words]
    print(lowered)
    corpus_stem.append(lowered)

['hollywood', 'home', 'minut', 'everyth', 'spacious', 'and', 'free', 'park', 'sustain', 'condo', 'hotel', 'los', 'angel', '2', 'mile', 'capitol', 'record', 'build', 'properti', 'encourag', 'guest', 'embrac', 'natur', 'ecofriend', 'arrang', 'condo', 'hotel', 'offer', 'free', 'privat', 'park', '24hour', 'front', 'desk', 'free', 'wifi', 'condo', 'hotel', 'privat', 'entranc', 'condo', 'hotel', 'unit', 'equip', 'air', 'condit', 'seat', 'area', 'flatscreen', 'tv', 'stream', 'servic', 'kitchen', 'dine', 'area', 'privat', 'bathroom', 'hair', 'dryer', 'bath', 'free', 'toiletri', 'dishwash', 'oven', 'microwav', 'also', 'offer', 'well', 'coffe', 'machin', 'kettl', 'condo', 'hotel', 'unit', 'come', 'bed', 'linen', 'towel', 'condo', 'hotel', 'picnic', 'area', 'spend', 'day', 'outdoor', 'dolbi', 'theater', '29', 'mile', 'condo', 'hotel', 'hollywood', 'bowl', '31', 'mile', 'properti', 'nearest', 'airport', 'hollywood', 'burbank', 'airport', '87', 'mile', 'hollywood', 'home', 'minut', 'everyth', 'spac

In [95]:
vocab, index = {}, 1  # start indexing from 1
for doc in corpus_stem:
    for token in doc:
        if token not in vocab:
            vocab[token] = index
            index += 1
        
vocab_stem=vocab
vocab_stem_size = len(vocab)
print(vocab_stem)
print(" ")
print("Total size of stemmed vocabulary:", vocab_stem_size)

{'hollywood': 1, 'home': 2, 'minut': 3, 'everyth': 4, 'spacious': 5, 'and': 6, 'free': 7, 'park': 8, 'sustain': 9, 'condo': 10, 'hotel': 11, 'los': 12, 'angel': 13, '2': 14, 'mile': 15, 'capitol': 16, 'record': 17, 'build': 18, 'properti': 19, 'encourag': 20, 'guest': 21, 'embrac': 22, 'natur': 23, 'ecofriend': 24, 'arrang': 25, 'offer': 26, 'privat': 27, '24hour': 28, 'front': 29, 'desk': 30, 'wifi': 31, 'entranc': 32, 'unit': 33, 'equip': 34, 'air': 35, 'condit': 36, 'seat': 37, 'area': 38, 'flatscreen': 39, 'tv': 40, 'stream': 41, 'servic': 42, 'kitchen': 43, 'dine': 44, 'bathroom': 45, 'hair': 46, 'dryer': 47, 'bath': 48, 'toiletri': 49, 'dishwash': 50, 'oven': 51, 'microwav': 52, 'also': 53, 'well': 54, 'coffe': 55, 'machin': 56, 'kettl': 57, 'come': 58, 'bed': 59, 'linen': 60, 'towel': 61, 'picnic': 62, 'spend': 63, 'day': 64, 'outdoor': 65, 'dolbi': 66, 'theater': 67, '29': 68, 'bowl': 69, '31': 70, 'nearest': 71, 'airport': 72, 'burbank': 73, '87': 74, 'overlook': 75, 'pacif': 

In [96]:
def vectorize(tokens, vocab):
    vector=[]
    for w in vocab:
        vector.append(tokens.count(w))
    return vector

vectors=[]
for doc in corpus_stem:
    vectors.append(vectorize(doc, vocab_stem))

In [97]:
#making a pandas dataframe
df = pd.DataFrame(vectors)
df

,0,1,2,3,4,5,6,7,8,9,...,2161,2162,2163,2164,2165,2166,2167,2168,2169,2170
0,4,2,2,2,2,2,5,3,1,7,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,3,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0
293,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
294,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
295,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [106]:
from sklearn.feature_extraction.text import CountVectorizer

data_copy = pd.read_csv('2024-03-03_2024-03-10_Los Ángeles, California, Estados Unidos.csv')
data_copy = clean_df(data_copy)
data_copy['description'] = data_copy['description'].apply(remove_info)

sentences = data_copy['description'].tolist()

cv = CountVectorizer(ngram_range = (1,2), min_df=0.05, max_df=0.5)
cv.fit(sentences)
vectorized_text=cv.transform(sentences)
vectorized_text=vectorized_text.todense()
print("document term matrix has size", vectorized_text.shape)
print(cv.get_feature_names_out())

document term matrix has size (297, 635)
['10' '10 minute' '10 minutes' '11' '11 mi' '11 miles' '12' '15'
 '15 minutes' '20' '24' '24 hour' 'access' 'access to' 'accommodation'
 'accommodation while' 'accommodations' 'accommodations and'
 'accommodations with' 'air' 'air conditioned' 'air conditioning'
 'airport 11' 'airport miles' 'all' 'all guest' 'all rooms' 'along'
 'along with' 'also available' 'also provided' 'amenities' 'american'
 'an on' 'an outdoor' 'an oven' 'and 24' 'and an' 'and bar' 'and coffee'
 'and dinner' 'and free' 'and guests' 'and hairdryer' 'and is' 'and mi'
 'and microwave' 'and miles' 'and private' 'and santa' 'and terrace'
 'and the' 'and towels' 'and work' 'angeles convention' 'angeles county'
 'angeles hotel' 'angeles international' 'angeles is' 'angeles miles'
 'angeles the' 'are also' 'are available' 'are equipped' 'are offered'
 'are provided' 'are within' 'area' 'area and' 'around' 'around miles'
 'art' 'art lacma' 'as' 'as well' 'at this' 'attractions' '

In [107]:
for document in vectorized_text:
    print(document)

[[0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1
  0 0 0 0 0 4 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 1 0 1 0 1 0 0 0 3 1 0 0 0
  0 2 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0
  1 1 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
  0 1 1 0 0 0 0 0 0 1 7 7 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 1
  1 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 2 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 1 0 0 2 0 0 0 0 0 0 0 0 0 4 1 1 0 0 0 0 0 0 0 0 0 0
  2 0 0 1 0 1 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 1 0 0 0 0 0 2 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0
  0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 4 0 0 0 0 0 0 2 0 0 0
  0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 1 0 0 0 0 3 1 0 0 0 0 0 0 0 0 0 3 1 1 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [109]:
from sklearn.feature_extraction.text import TfidfVectorizer

cv = TfidfVectorizer(ngram_range = (1,2), min_df=0.05, max_df=0.5, norm=None)
cv.fit(sentences)
vectorized_text=cv.transform(sentences)
vectorized_text=vectorized_text.todense()
print("document term matrix has size", vectorized_text.shape)
print(cv.get_feature_names_out())

document term matrix has size (297, 635)
['10' '10 minute' '10 minutes' '11' '11 mi' '11 miles' '12' '15'
 '15 minutes' '20' '24' '24 hour' 'access' 'access to' 'accommodation'
 'accommodation while' 'accommodations' 'accommodations and'
 'accommodations with' 'air' 'air conditioned' 'air conditioning'
 'airport 11' 'airport miles' 'all' 'all guest' 'all rooms' 'along'
 'along with' 'also available' 'also provided' 'amenities' 'american'
 'an on' 'an outdoor' 'an oven' 'and 24' 'and an' 'and bar' 'and coffee'
 'and dinner' 'and free' 'and guests' 'and hairdryer' 'and is' 'and mi'
 'and microwave' 'and miles' 'and private' 'and santa' 'and terrace'
 'and the' 'and towels' 'and work' 'angeles convention' 'angeles county'
 'angeles hotel' 'angeles international' 'angeles is' 'angeles miles'
 'angeles the' 'are also' 'are available' 'are equipped' 'are offered'
 'are provided' 'are within' 'area' 'area and' 'around' 'around miles'
 'art' 'art lacma' 'as' 'as well' 'at this' 'attractions' '

In [110]:
vectorized_text=vectorized_text.round(decimals=3, out=None)
for document in vectorized_text:
    print(document)

[ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  1.853  1.97   0.     0.     0.     0.     0.     0.     0.     1.792
  0.     3.114  0.     2.554  0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     3.864  0.     0.     0.     0.
  0.     9.91   0.     0.     0.     0.     3.864  0.     3.086  0.
  0.     0.     2.936  0.     0.     0.     0.     0.     0.     2.89
  0.     2.538  0.     2.913  0.     0.     0.     6.694  3.753  0.
  0.     0.     0.     5.14   2.847  0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     3.365
  2.22   3.201  0.     0.     0.     0.     0.     0.     2.554  2.746
  0.     0.     0.     0.     0.     0.     0.     0.     2.746  3.171
  0.     0.     2.603  3.231  3.231  0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     2.69   2.69   0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     2.062  2.826  0.  

In [260]:
data_c1_w1 = pd.read_csv('2024-03-03_2024-03-10_LA_cleaned.csv')
data_c1_w2 = pd.read_csv('2024-04-14_2024-04-21_LA_cleaned.csv')
data_c2_w1 = pd.read_csv('2024-03-03_2024-03-10_SF_cleaned.csv')
data_c2_w2 = pd.read_csv('2024-04-14_2024-04-21_SF_cleaned.csv')
data_c1_w1['review_count'] = pd.to_numeric(data_c1_w1['review_count'].str.replace(r'[a-zA-Z+,]','', regex=True))
data_c1_w2['review_count'] = pd.to_numeric(data_c1_w2['review_count'].str.replace(r'[a-zA-Z+,]','', regex=True))
data_c2_w1['review_count'] = pd.to_numeric(data_c2_w1['review_count'].str.replace(r'[a-zA-Z+,]','', regex=True))
data_c2_w2['review_count'] = pd.to_numeric(data_c2_w2['review_count'].str.replace(r'[a-zA-Z+,]','', regex=True))


In [261]:
data_c1_w1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297 entries, 0 to 296
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0.1       297 non-null    int64  
 1   Unnamed: 0         297 non-null    int64  
 2   photo_url          297 non-null    object 
 3   link               297 non-null    object 
 4   name               297 non-null    object 
 5   central_distance   297 non-null    float64
 6   sustainability     153 non-null    float64
 7   rating             286 non-null    float64
 8   comfort_rating     207 non-null    float64
 9   review_count       286 non-null    float64
 10  room_description   297 non-null    object 
 11  room_specifics     296 non-null    object 
 12  price              297 non-null    int64  
 13  original_price     111 non-null    float64
 14  neighborhood_city  297 non-null    object 
 15  start_date         297 non-null    object 
 16  end_date           297 non

In [262]:
data_c1_w1.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'photo_url', 'link', 'name',
       'central_distance', 'sustainability', 'rating', 'comfort_rating',
       'review_count', 'room_description', 'room_specifics', 'price',
       'original_price', 'neighborhood_city', 'start_date', 'end_date',
       'map_lat', 'map_lng', 'name.1', 'description'],
      dtype='object')

In [263]:
data_c1_w1.drop(columns=['Unnamed: 0', 'photo_url', 'link', 'name', 'original_price', 'map_lat', 'map_lng', 'name.1'], inplace=True)

In [264]:
data_c1_w1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297 entries, 0 to 296
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0.1       297 non-null    int64  
 1   central_distance   297 non-null    float64
 2   sustainability     153 non-null    float64
 3   rating             286 non-null    float64
 4   comfort_rating     207 non-null    float64
 5   review_count       286 non-null    float64
 6   room_description   297 non-null    object 
 7   room_specifics     296 non-null    object 
 8   price              297 non-null    int64  
 9   neighborhood_city  297 non-null    object 
 10  start_date         297 non-null    object 
 11  end_date           297 non-null    object 
 12  description        297 non-null    object 
dtypes: float64(5), int64(2), object(6)
memory usage: 30.3+ KB


In [265]:
num_feat = ['central_distance', 'sustainability', 'rating', 'comfort_rating', 'review_count']
cat_feat = ['room_description', 'room_specifics', 'neighborhood_city']

for c in cat_feat:
    print(f'{c}:')
    print(data_c1_w1[c].unique())

room_description:
['Studio' 'Epic Room King View Top Floor' 'One-Bedroom Apartment'
 'King Room' 'Queen Room with Two Queen Beds'
 'Urban King - Hearing Accessible (No Parking)' 'Junior Suite'
 'Standard King Room' 'King Suite' 'Standard Queen Room'
 'Economy Studio with Partial Ocean View' 'Select Queen Room'
 'Deluxe King Room' 'Signature King' 'Deluxe Queen Room' 'Chelsea Suite'
 'Queen Room with Shared Bathroom' 'Superior Queen Room'
 'King Room - Non-Smoking' 'Guest room, 1 King, Sofa bed, City view'
 'Essential King Room with Hollywood Sign View'
 'King Room with Courtyard View' 'Superior Room, 1 king-size bed'
 'Studio Suite - 1 Queen Bed' 'Queen Junior Suite'
 'Bedroom Deluxe Broadway Suite with One King Bed, Non-Smoking'
 'King or Queen Room' 'Double or Twin Room with Partial Sea View'
 'King Room - Hearing Accessible - Non-Smoking' 'Snug'
 'King Room with Sofa Bed' 'Superior King Room' 'Queen Room'
 'ADA Deluxe King Suite' '1 Queen Bed, Classic Guest Room'
 'King Room - Bronz

In [266]:
# Function to extract the first number from a text
def extract_first_number(text):
    # Regular expression pattern
    pattern = re.compile(r'\b\d+\b')

    try:
        match = pattern.search(text)
        return int(match.group()) if match else None
    except:
        return None

data_c1_w1['num_beds'] = data_c1_w1['room_specifics'].apply(lambda x: extract_first_number(x))
data_c1_w1.drop(columns=['room_description', 'neighborhood_city', 'room_specifics'], inplace=True)
data_c1_w1['city'] = 'Los Angeles'

In [267]:
data_c1_w1.sort_values('price', inplace=True)
for c in num_feat+['num_beds']:
    data_c1_w1[c].ffill(inplace=True)
    data_c1_w1[c].bfill(inplace=True)

data_c1_w1['num_beds'] = data_c1_w1['num_beds'].astype(int)

In [268]:
data_c1_w1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 297 entries, 198 to 86
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0.1      297 non-null    int64  
 1   central_distance  297 non-null    float64
 2   sustainability    297 non-null    float64
 3   rating            297 non-null    float64
 4   comfort_rating    297 non-null    float64
 5   review_count      297 non-null    float64
 6   price             297 non-null    int64  
 7   start_date        297 non-null    object 
 8   end_date          297 non-null    object 
 9   description       297 non-null    object 
 10  num_beds          297 non-null    int32  
 11  city              297 non-null    object 
dtypes: float64(5), int32(1), int64(2), object(4)
memory usage: 29.0+ KB


In [269]:
data_c1_w2['num_beds'] = data_c1_w2['room_specifics'].apply(lambda x: extract_first_number(x))
data_c1_w2.drop(columns=['Unnamed: 0', 'photo_url', 'link', 'name', 'original_price', 'map_lat', 'map_lng', 'name.1', 'room_description', 'neighborhood_city', 'room_specifics'], inplace=True)
data_c1_w2['city'] = 'Los Angeles'
data_c1_w2.sort_values('price', inplace=True)
for c in num_feat+['num_beds']:
    data_c1_w2[c].ffill(inplace=True)
    data_c1_w2[c].bfill(inplace=True)

data_c1_w2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 298 entries, 153 to 84
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0.1      298 non-null    int64  
 1   central_distance  298 non-null    float64
 2   sustainability    298 non-null    float64
 3   rating            298 non-null    float64
 4   comfort_rating    298 non-null    float64
 5   review_count      298 non-null    float64
 6   price             298 non-null    int64  
 7   start_date        298 non-null    object 
 8   end_date          298 non-null    object 
 9   description       298 non-null    object 
 10  num_beds          298 non-null    float64
 11  city              298 non-null    object 
dtypes: float64(6), int64(2), object(4)
memory usage: 30.3+ KB


In [270]:
data_c2_w1['num_beds'] = data_c2_w1['room_specifics'].apply(lambda x: extract_first_number(x))
data_c2_w1.drop(columns=['Unnamed: 0', 'photo_url', 'link', 'name', 'original_price', 'map_lat', 'map_lng', 'name.1', 'room_description', 'neighborhood_city', 'room_specifics'], inplace=True)
data_c2_w1['city'] = 'San Francisco'
data_c2_w1.sort_values('price', inplace=True)
for c in num_feat+['num_beds']:
    data_c2_w1[c].ffill(inplace=True)
    data_c2_w1[c].bfill(inplace=True)

data_c2_w1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 188 entries, 43 to 134
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0.1      188 non-null    int64  
 1   central_distance  188 non-null    float64
 2   sustainability    188 non-null    float64
 3   rating            188 non-null    float64
 4   comfort_rating    188 non-null    float64
 5   review_count      188 non-null    float64
 6   price             188 non-null    int64  
 7   start_date        188 non-null    object 
 8   end_date          188 non-null    object 
 9   description       188 non-null    object 
 10  num_beds          188 non-null    int64  
 11  city              188 non-null    object 
dtypes: float64(5), int64(3), object(4)
memory usage: 19.1+ KB


In [271]:
data_c2_w2['num_beds'] = data_c2_w2['room_specifics'].apply(lambda x: extract_first_number(x))
data_c2_w2.drop(columns=['Unnamed: 0', 'photo_url', 'link', 'name', 'original_price', 'map_lat', 'map_lng', 'name.1', 'room_description', 'neighborhood_city', 'room_specifics'], inplace=True)
data_c2_w2['city'] = 'San Francisco'
data_c2_w2.sort_values('price', inplace=True)
for c in num_feat+['num_beds']:
    data_c2_w2[c].ffill(inplace=True)
    data_c2_w2[c].bfill(inplace=True)

data_c2_w2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185 entries, 96 to 101
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0.1      185 non-null    int64  
 1   central_distance  185 non-null    float64
 2   sustainability    185 non-null    float64
 3   rating            185 non-null    float64
 4   comfort_rating    185 non-null    float64
 5   review_count      185 non-null    float64
 6   price             185 non-null    int64  
 7   start_date        185 non-null    object 
 8   end_date          185 non-null    object 
 9   description       185 non-null    object 
 10  num_beds          185 non-null    float64
 11  city              185 non-null    object 
dtypes: float64(6), int64(2), object(4)
memory usage: 18.8+ KB


In [272]:
# First equation
data_c1_w1['event'] = 0
data_c1_w2['event'] = 1

data_eq1 = pd.concat([data_c1_w1, data_c1_w2]).drop(columns=['city', 'start_date', 'end_date', 'description'])
data_eq1.head()

,Unnamed: 0.1,central_distance,sustainability,rating,comfort_rating,review_count,price,num_beds,event
198,198,3.3,1.0,5.0,8.2,169.0,519,1.0,0
222,222,9.5,1.0,7.4,8.2,5.0,541,1.0,0
127,127,17.1,1.0,6.6,8.2,1865.0,662,1.0,0
23,23,19.6,1.0,7.4,8.2,2258.0,671,1.0,0
35,35,1.0,1.0,7.6,8.2,1644.0,686,1.0,0


In [273]:
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing

# Create a LinearRegression model
model = LinearRegression()
features = ['central_distance', 'sustainability', 'rating', 'comfort_rating',
    'review_count', 'num_beds', 'event']

# scale continuous variables
scaler = preprocessing.StandardScaler()
scaler.fit(data_eq1[features])## Fitted to the training set

X_data = scaler.transform(data_eq1[features])
X_data=pd.DataFrame(X_data, columns=features)

# Fit the model to the data_eq1
model.fit(X_data, data_eq1['price'])

# Extract the coefficients
beta_0 = round(model.intercept_, 2)
beta_1 = round(model.coef_[0], 2)
beta_2 = round(model.coef_[1], 2)
beta_3 = round(model.coef_[2], 2)
beta_4 = round(model.coef_[3], 2)
beta_5 = round(model.coef_[4], 2)
beta_6 = round(model.coef_[5], 2)
beta_7 = round(model.coef_[6], 2)

print("Beta_0:", beta_0)
print("Beta for  central distance: ", beta_1)
print("Beta for sustainability: ", beta_2)
print("Beta for rating: ", beta_3)
print("Beta for comfort rating: ", beta_4)
print("Beta for review count: ", beta_5)
print("Betas for number of beds: ", beta_6)
print("Betas for dummy event: ", beta_7)

Beta_0: 2045.68
Beta for  central distance:  -59.19
Beta for sustainability:  122.28
Beta for rating:  440.37
Beta for comfort rating:  354.04
Beta for review count:  -216.83
Betas for number of beds:  -31.6
Betas for dummy event:  38.72


In [274]:
import numpy as np

# Second equation
data_c1_w1.drop(columns=['event'], inplace=True)
data_c1_w2.drop(columns=['event'], inplace=True)

data_eq2 = pd.concat([data_c1_w1, data_c2_w1]).drop(columns=['start_date', 'end_date', 'description'])
data_eq2['dummy_city'] = np.where(data_eq2['city'] == 'Los Angeles',1 ,0)
data_eq2.drop(columns=['city'], inplace=True)
data_eq2.head()

,Unnamed: 0.1,central_distance,sustainability,rating,comfort_rating,review_count,price,num_beds,dummy_city
198,198,3.3,1.0,5.0,8.2,169.0,519,1,1
222,222,9.5,1.0,7.4,8.2,5.0,541,1,1
127,127,17.1,1.0,6.6,8.2,1865.0,662,1,1
23,23,19.6,1.0,7.4,8.2,2258.0,671,1,1
35,35,1.0,1.0,7.6,8.2,1644.0,686,1,1


In [275]:
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing

# Create a LinearRegression model
model = LinearRegression()
features = ['central_distance', 'sustainability', 'rating', 'comfort_rating',
    'review_count', 'num_beds', 'dummy_city']

# scale continuous variables
scaler = preprocessing.StandardScaler()
scaler.fit(data_eq2[features])## Fitted to the training set

X_data = scaler.transform(data_eq2[features])
X_data=pd.DataFrame(X_data, columns=features)

# Fit the model to the data_eq1
model.fit(X_data, data_eq2['price'])

# Extract the coefficients
beta_0 = round(model.intercept_, 2)
beta_1 = round(model.coef_[0], 2)
beta_2 = round(model.coef_[1], 2)
beta_3 = round(model.coef_[2], 2)
beta_4 = round(model.coef_[3], 2)
beta_5 = round(model.coef_[4], 2)
beta_6 = round(model.coef_[5], 2)
beta_7 = round(model.coef_[6], 2)

print("Beta_0:", beta_0)
print("Beta for  central distance: ", beta_1)
print("Beta for sustainability: ", beta_2)
print("Beta for rating: ", beta_3)
print("Beta for comfort rating: ", beta_4)
print("Beta for review count: ", beta_5)
print("Betas for number of beds: ", beta_6)
print("Betas for dummy city: ", beta_7)

Beta_0: 2110.23
Beta for  central distance:  -195.23
Beta for sustainability:  125.76
Beta for rating:  394.6
Beta for comfort rating:  431.07
Beta for review count:  -102.47
Betas for number of beds:  301.46
Betas for dummy city:  -2.06


In [276]:
# Third equation
data_c1_w1['event'] = 0
data_c1_w2['event'] = 1
data_c2_w1['event'] = 0
data_c2_w2['event'] = 1

data_eq3 = pd.concat([data_c1_w1, data_c2_w1, data_c2_w1, data_c2_w2]).drop(columns=['start_date', 'end_date', 'description'])
data_eq3['dummy_city'] = np.where(data_eq3['city'] == 'Los Angeles',1 ,0)
data_eq3.drop(columns=['city'], inplace=True)
data_eq3.head()

,Unnamed: 0.1,central_distance,sustainability,rating,comfort_rating,review_count,price,num_beds,event,dummy_city
198,198,3.3,1.0,5.0,8.2,169.0,519,1.0,0,1
222,222,9.5,1.0,7.4,8.2,5.0,541,1.0,0,1
127,127,17.1,1.0,6.6,8.2,1865.0,662,1.0,0,1
23,23,19.6,1.0,7.4,8.2,2258.0,671,1.0,0,1
35,35,1.0,1.0,7.6,8.2,1644.0,686,1.0,0,1


In [277]:
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing

# Create a LinearRegression model
model = LinearRegression()
features = ['central_distance', 'sustainability', 'rating', 'comfort_rating',
    'review_count', 'num_beds', 'event', 'dummy_city']

# scale continuous variables
scaler = preprocessing.StandardScaler()
scaler.fit(data_eq3[features])## Fitted to the training set

X_data = scaler.transform(data_eq3[features])
X_data=pd.DataFrame(X_data, columns=features)

# Fit the model to the data_eq1
model.fit(X_data, data_eq3['price'])

# Extract the coefficients
beta_0 = round(model.intercept_, 2)
beta_1 = round(model.coef_[0], 2)
beta_2 = round(model.coef_[1], 2)
beta_3 = round(model.coef_[2], 2)
beta_4 = round(model.coef_[3], 2)
beta_5 = round(model.coef_[4], 2)
beta_6 = round(model.coef_[5], 2)
beta_7 = round(model.coef_[6], 2)
beta_8 = round(model.coef_[7], 2)

print("Beta_0:", beta_0)
print("Beta for  central distance: ", beta_1)
print("Beta for sustainability: ", beta_2)
print("Beta for rating: ", beta_3)
print("Beta for comfort rating: ", beta_4)
print("Beta for review count: ", beta_5)
print("Betas for number of beds: ", beta_6)
print("Betas for dummy event: ", beta_7)
print("Betas for dummy city: ", beta_8)

Beta_0: 1985.93
Beta for  central distance:  -179.24
Beta for sustainability:  85.64
Beta for rating:  320.36
Beta for comfort rating:  480.47
Beta for review count:  -79.92
Betas for number of beds:  351.03
Betas for dummy event:  -104.92
Betas for dummy city:  18.76
